In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Custom functions to create visual charts
from chart_utils import TimeSeriesImageDataset, create_area_chart, create_bar_chart

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read UCR Dataset
def read_ucr(filename):
    data = np.loadtxt(filename, delimiter=',')
    Y = data[:, 0]
    X = data[:, 1:]
    return X, Y

# File paths
train_file = 'Adiac/Adiac_TRAIN'
test_file = 'Adiac/Adiac_TEST'

# Load dataset
x_train, y_train = read_ucr(train_file)
x_test, y_test = read_ucr(test_file)

# Normalize labels to be within range [0, num_classes-1]
nb_classes = len(np.unique(y_test))
y_train = (y_train - y_train.min()).astype(int)
y_test = (y_test - y_test.min()).astype(int)

# Normalize features
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

# Create dataset and dataloader
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])
train_dataset = TimeSeriesImageDataset(x_train, y_train, transform)
test_dataset = TimeSeriesImageDataset(x_test, y_test, transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define SimpleCNN
class SimpleCNN(nn.Module):
    def __init__(self, input_channels):
        super(SimpleCNN, self).__init__()
        self.layer_stack = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )

    def forward(self, x):
        return self.layer_stack(x)

# Define CombinedModel
class CombinedModel(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(CombinedModel, self).__init__()
        self.cnn_area = SimpleCNN(input_shape[0])
        self.cnn_bar = SimpleCNN(input_shape[0])

        # Calculate the flattened feature size
        cnn_output_size = self._get_cnn_output_size(input_shape)

        # Fully connected layers
        self.fc1 = nn.Sequential(
            nn.Linear(cnn_output_size * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )
        
    # Flattened output size of CNN
    def _get_cnn_output_size(self, input_shape):
        dummy_input = torch.zeros(1, *input_shape).to(device)
        output = self.cnn_area(dummy_input)
        return output.view(1, -1).size(1)

    def forward(self, x_area, x_bar):
        features_area = self.cnn_area(x_area)
        features_bar = self.cnn_bar(x_bar)

        concatenated_features = torch.cat((features_area, features_bar), dim=1)

        return self.fc1(concatenated_features)

# Example usage of the model
input_shape = (3, 128, 128)  # Example input shape (channels, height, width)
num_classes = nb_classes  # Number of output classes

model = CombinedModel(input_shape, num_classes).to(device)
print(model)

# Training loop with accuracy and classified output
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

num_epochs = 5
overall_progress_bar = tqdm(range(num_epochs), desc="Training Progress")
for epoch in overall_progress_bar:
    print(f"Epoch: {epoch+1}/{num_epochs}\n-------")
    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []

    # Training phase
    model.train()
    for batch, (images_area, images_bar, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()

        # Move data to device
        images_area, images_bar, labels = images_area.to(device), images_bar.to(device), labels.to(device)

        # Convert labels to LongTensor
        labels = labels.long()

        outputs = model(images_area, images_bar)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Collect labels and predictions for output
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

    # Divide total train loss by length of train dataloader (average loss per batch per epoch)
    epoch_loss = running_loss / len(train_dataloader)
    epoch_accuracy = 100 * correct / total

    print(f'Training Loss: {epoch_loss:.4f} | Training Accuracy: {epoch_accuracy:.2f}%')

    # Testing phase
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.inference_mode():
        for images_area, images_bar, labels in test_dataloader:
            # Move data to device
            images_area, images_bar, labels = images_area.to(device), images_bar.to(device), labels.to(device)
            
            labels = labels.long()
            outputs = model(images_area, images_bar)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss /= len(test_dataloader)
    test_accuracy = 100 * correct / total

    print(f'Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%')

    # Step the scheduler
    scheduler.step(test_loss)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.


CombinedModel(
  (cnn_area): SimpleCNN(
    (layer_stack): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (12): Flatten(start_dim=1, end_dim=-1)
    )
  )
  (cnn_bar): SimpleCNN(
    (layer_stack): Sequential(


Training Progress:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1/5
-------
Training Loss: 4.8156 | Training Accuracy: 3.85%


Training Progress:  20%|██        | 1/5 [00:31<02:06, 31.65s/it]

...

Test Loss: 3.6198 | Test Accuracy: 3.84%
Epoch: 4/5
-------
Training Loss: 3.6078 | Training Accuracy: 3.08%


Training Progress:  80%|████████  | 4/5 [02:06<00:31, 31.66s/it]

Test Loss: 3.6164 | Test Accuracy: 1.02%
Epoch: 5/5
-------
Training Loss: 3.6109 | Training Accuracy: 4.62%


Training Progress: 100%|██████████| 5/5 [02:38<00:00, 31.67s/it]

Test Loss: 3.6160 | Test Accuracy: 3.07%
